# Optimization Overview

## Overview
This notebook provides a high-level tour of the insurance optimization algorithms available in the framework. We compare optimization methods, quantify the ergodic advantage over traditional ensemble approaches, and measure the impact of insurance on key business metrics.

- **Prerequisites**: [core/01_ergodic_foundations](../core/01_ergodic_foundations.ipynb), [core/03_monte_carlo_simulation](../core/03_monte_carlo_simulation.ipynb)
- **Estimated runtime**: 2-3 minutes
- **Audience**: [Practitioner]

## Setup

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time

from ergodic_insurance import ManufacturerConfig, InsuranceProgram, EnhancedInsuranceLayer
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.loss_distributions import ManufacturingLossGenerator
from ergodic_insurance.monte_carlo import MonteCarloEngine
from ergodic_insurance.ergodic_analyzer import ErgodicAnalyzer
from ergodic_insurance.decision_engine import (
    InsuranceDecisionEngine,
    DecisionOptimizationConstraints,
    OptimizationMethod,
)
from ergodic_insurance.visualization import StyleManager, Theme

# Reproducibility
SEED = 42
np.random.seed(SEED)

## 1. Algorithm Performance Comparison

The `InsuranceDecisionEngine` supports several optimization methods.  We compare three here:

| Method | Characteristics |
|---|---|
| **SLSQP** | Fast, gradient-based local optimizer |
| **Differential Evolution** | Slower but globally-searching evolutionary method |
| **Weighted Sum** | Multi-objective scalarization approach |

> **Note**: The cell below runs actual optimizations.  Results will vary by
> seed and hardware speed; the relative ordering is the main takeaway.

In [ ]:
# --- Manufacturer & loss setup (shared across sections) ---
manufacturer_config = ManufacturerConfig(
    initial_assets=50_000_000,
    asset_turnover_ratio=1.2,
    base_operating_margin=0.15,
    tax_rate=0.25,
    retention_ratio=0.7,
)
manufacturer = WidgetManufacturer(manufacturer_config)
expected_revenue = float(manufacturer.calculate_revenue())
revenue_scale = expected_revenue / 10_000_000

loss_generator = ManufacturingLossGenerator(
    attritional_params={
        "base_frequency": 3.0 * revenue_scale,
        "severity_mean": 25_000,
        "severity_cv": 0.6,
    },
    large_params={
        "base_frequency": 0.3 * revenue_scale,
        "severity_mean": 500_000,
        "severity_cv": 0.8,
    },
    catastrophic_params={
        "base_frequency": 0.01 * revenue_scale,
        "severity_xm": 5_000_000,
        "severity_alpha": 2.0,
    },
    seed=SEED,
)

# Estimate expected annual loss for premium budgeting
annual_losses = [
    loss_generator.generate_losses(duration=1.0, revenue=expected_revenue)[1]["total_amount"]
    for _ in range(1_000)
]
expected_annual_loss = float(np.mean(annual_losses))

print(f"Expected annual revenue : ${expected_revenue:>14,.0f}")
print(f"Expected annual loss    : ${expected_annual_loss:>14,.0f}")
print(f"Loss / Revenue ratio    : {expected_annual_loss / expected_revenue:>14.2%}")

In [ ]:
# Run each optimization method and record wall-clock time + result
max_premium = expected_annual_loss / 0.7 * 1.5

constraints = DecisionOptimizationConstraints(
    max_premium_budget=max_premium,
    min_coverage_limit=1_000_000,
    max_coverage_limit=20_000_000,
    max_bankruptcy_probability=0.05,
    min_retained_limit=50_000,
    max_retained_limit=2_000_000,
)

engine = InsuranceDecisionEngine(
    manufacturer=manufacturer,
    loss_distribution=loss_generator,
    pricing_scenario="baseline",
)

methods = [
    OptimizationMethod.SLSQP,
    OptimizationMethod.DIFFERENTIAL_EVOLUTION,
    OptimizationMethod.WEIGHTED_SUM,
]

algo_results = []
for method in methods:
    t0 = time.time()
    decision = engine.optimize_insurance_decision(method=method, constraints=constraints)
    elapsed = time.time() - t0

    metrics = engine.calculate_decision_metrics(decision)

    algo_results.append({
        "method": method.value,
        "execution_time": elapsed,
        "ergodic_growth": metrics.ergodic_growth_rate,
        "bankruptcy_prob": metrics.bankruptcy_probability,
        "expected_roe": metrics.expected_roe,
        "total_premium": decision.total_premium,
        "total_coverage": decision.total_coverage,
        "premium_to_loss": decision.total_premium / expected_annual_loss if expected_annual_loss > 0 else 0,
    })
    print(f"{method.value:>25s}  {elapsed:5.1f}s  "
          f"growth={metrics.ergodic_growth_rate:.2%}  "
          f"ruin={metrics.bankruptcy_probability:.2%}")

algo_df = pd.DataFrame(algo_results)

In [ ]:
# Visualize algorithm comparison
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=("Execution Time (s)", "Ergodic Growth Rate (%)", "Bankruptcy Probability (%)"),
)

for i, (col, fmt) in enumerate(
    [("execution_time", ".1f"), ("ergodic_growth", ".1%"), ("bankruptcy_prob", ".2%")], start=1
):
    vals = algo_df[col] * (100 if "%" in fmt else 1)
    fig.add_trace(
        go.Bar(x=algo_df["method"], y=vals, text=[f"{v:{fmt}}" for v in algo_df[col]], textposition="outside"),
        row=1, col=i,
    )

fig.update_layout(height=400, showlegend=False, template="plotly_white",
                  title_text="Optimization Algorithm Comparison")
fig.show()

## 2. Ergodic vs. Ensemble Benefit Quantification

We compare three insurance strategies to measure the ergodic advantage:

| Strategy | Description |
|---|---|
| No Insurance | Self-insure all risks |
| Traditional (Ensemble) | Minimize expected cost |
| Ergodic Optimal | Maximize time-average growth rate |

In [ ]:
# Define three insurance structures
scenarios = {
    "No Insurance": InsuranceProgram([]),
    "Traditional (Ensemble)": InsuranceProgram([
        EnhancedInsuranceLayer(
            attachment_point=100_000, limit=2_000_000,
            base_premium_rate=(expected_annual_loss * 0.4) / expected_revenue,
        ),
        EnhancedInsuranceLayer(
            attachment_point=2_100_000, limit=3_000_000,
            base_premium_rate=(expected_annual_loss * 0.2) / expected_revenue / 2,
        ),
    ]),
    "Ergodic Optimal": InsuranceProgram([
        EnhancedInsuranceLayer(
            attachment_point=50_000, limit=1_500_000,
            base_premium_rate=(expected_annual_loss * 0.5) / expected_revenue,
        ),
        EnhancedInsuranceLayer(
            attachment_point=1_550_000, limit=2_500_000,
            base_premium_rate=(expected_annual_loss * 0.3) / expected_revenue / 1.5,
        ),
        EnhancedInsuranceLayer(
            attachment_point=4_050_000, limit=6_000_000,
            base_premium_rate=(expected_annual_loss * 0.1) / expected_revenue / 3,
        ),
    ]),
}

N_SIMS = 500
N_YEARS = 10

scenario_results = []
for name, program in scenarios.items():
    premium = float(program.calculate_annual_premium())
    mfg = WidgetManufacturer(manufacturer_config)

    final_assets = []
    for _ in range(N_SIMS):
        m = WidgetManufacturer(manufacturer_config)
        survived = True
        for yr in range(N_YEARS):
            rev = float(m.calculate_revenue())
            _, stats = loss_generator.generate_losses(1.0, rev)
            recovery = program.process_claim(stats["total_amount"])
            net_loss = stats["total_amount"] - recovery["insurance_recovery"]
            if net_loss > 0:
                m.process_insurance_claim(net_loss)
            if premium > 0:
                m.record_insurance_premium(premium)
            m.step(growth_rate=0.0)
            if float(m.equity) <= 0 or float(m.total_assets) <= 0:
                survived = False
                break
        final_assets.append(float(m.total_assets) if survived else 0)

    fa = np.array(final_assets)
    surviving = fa[fa > 0]
    if len(surviving) > 0:
        growth_rates = (surviving / manufacturer_config.initial_assets) ** (1 / N_YEARS) - 1
        ergodic_growth = np.mean(growth_rates)
    else:
        ergodic_growth = -1.0

    scenario_results.append({
        "scenario": name,
        "premium": premium,
        "ergodic_growth": ergodic_growth,
        "ruin_probability": np.mean(fa <= 0),
        "mean_final_assets": np.mean(fa),
    })
    print(f"{name:>25s}  growth={ergodic_growth:+.2%}  ruin={np.mean(fa <= 0):.1%}")

scenario_df = pd.DataFrame(scenario_results)

In [ ]:
# Visualize scenario comparison
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Ergodic Growth Rate", "Ruin Probability"),
)

colors = ["#e15759", "#f28e2b", "#59a14f"]

fig.add_trace(
    go.Bar(x=scenario_df["scenario"], y=scenario_df["ergodic_growth"] * 100,
           marker_color=colors,
           text=[f"{v:.1%}" for v in scenario_df["ergodic_growth"]], textposition="outside"),
    row=1, col=1,
)
fig.add_trace(
    go.Bar(x=scenario_df["scenario"], y=scenario_df["ruin_probability"] * 100,
           marker_color=colors,
           text=[f"{v:.1%}" for v in scenario_df["ruin_probability"]], textposition="outside"),
    row=1, col=2,
)

fig.update_yaxes(title_text="Growth Rate (%)", row=1, col=1)
fig.update_yaxes(title_text="Ruin Probability (%)", row=1, col=2)
fig.update_layout(height=400, showlegend=False, template="plotly_white",
                  title_text="Ergodic vs Traditional Insurance Optimization")
fig.show()

## 3. Business Metrics Impact

How does increasing insurance coverage affect ROE, bankruptcy risk, and risk-adjusted returns?

In [ ]:
# Sweep over five coverage levels
coverage_fractions = [0.0, 0.25, 0.50, 0.75, 1.0]
coverage_names = ["None", "Minimal", "Standard", "Enhanced", "Comprehensive"]

biz_results = []
for frac, cname in zip(coverage_fractions, coverage_names):
    if frac == 0:
        layers = []
    else:
        covered = expected_annual_loss * frac
        layers = [
            EnhancedInsuranceLayer(
                attachment_point=int(100_000 * (1 - frac) + 10_000),
                limit=int(2_000_000 * frac + 500_000),
                base_premium_rate=(covered * 0.6 / 0.7) / expected_revenue,
            ),
            EnhancedInsuranceLayer(
                attachment_point=int(2_000_000 * frac + 600_000),
                limit=int(3_000_000 * frac + 500_000),
                base_premium_rate=(covered * 0.4 / 0.7) / expected_revenue / 2,
            ),
        ]

    program = InsuranceProgram(layers)
    premium = float(program.calculate_annual_premium())

    returns = []
    bankruptcies = 0
    for _ in range(N_SIMS):
        m = WidgetManufacturer(manufacturer_config)
        init_eq = float(m.equity)
        survived = True
        for yr in range(N_YEARS):
            rev = float(m.calculate_revenue())
            _, stats = loss_generator.generate_losses(1.0, rev)
            recovery = program.process_claim(stats["total_amount"])
            net_loss = stats["total_amount"] - recovery["insurance_recovery"]
            if net_loss > 0:
                m.process_insurance_claim(net_loss)
            if premium > 0:
                m.record_insurance_premium(premium)
            m.step(growth_rate=0.0)
            if float(m.equity) <= 0:
                bankruptcies += 1
                survived = False
                break
        if survived:
            returns.append((float(m.equity) / init_eq) ** (1 / N_YEARS) - 1)

    mean_roe = np.mean(returns) if returns else -1.0
    std_roe = np.std(returns) if returns else 0.0
    sharpe = mean_roe / std_roe if std_roe > 0 else 0.0

    biz_results.append({
        "level": cname,
        "coverage_frac": frac,
        "premium": premium,
        "mean_roe": mean_roe,
        "bankruptcy_rate": bankruptcies / N_SIMS,
        "sharpe": sharpe,
    })

biz_df = pd.DataFrame(biz_results)
print(biz_df[["level", "premium", "mean_roe", "bankruptcy_rate", "sharpe"]].to_string(index=False))

In [ ]:
# Visualize business impact
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=("Mean ROE (%)", "Bankruptcy Rate (%)", "Sharpe Ratio"),
)

for i, col in enumerate(["mean_roe", "bankruptcy_rate", "sharpe"], start=1):
    scale = 100 if col != "sharpe" else 1
    fig.add_trace(
        go.Bar(x=biz_df["level"], y=biz_df[col] * scale,
               text=[f"{v:.1%}" if col != "sharpe" else f"{v:.2f}" for v in biz_df[col]],
               textposition="outside"),
        row=1, col=i,
    )

fig.update_layout(height=400, showlegend=False, template="plotly_white",
                  title_text="Business Metrics by Insurance Level")
fig.show()

## Key Takeaways

- **Differential Evolution** tends to find the best global optimum but takes longer; **SLSQP** is a good fast alternative for quick iteration.
- **Ergodic (time-average) optimization** delivers materially higher long-term growth than ensemble-based strategies, especially under catastrophic tail risk.
- Increasing coverage generally **reduces bankruptcy risk** and can **improve risk-adjusted ROE** (Sharpe ratio), even though raw premium costs rise.
- The ergodic framework transforms the insurance purchase from a "cost to minimize" into a "growth lever to optimize."

## Next Steps

- [optimization/02_sensitivity_analysis](02_sensitivity_analysis.ipynb) -- understand which parameters matter most
- [optimization/03_pareto_analysis](03_pareto_analysis.ipynb) -- explore multi-objective trade-offs
- [optimization/04_retention_optimization](04_retention_optimization.ipynb) -- deep-dive into optimal deductible selection